In [17]:
LANGHCHAIN_TRACINB_V2 = "true"
LANGCHAIN_API_KEY = "ls__ef7c69da49cf48719e4d7c4b33a3debd"
import os
os.environ['LANGCHAIN_API_KEY'] = LANGCHAIN_API_KEY

In [18]:
import json

with open("train.json", "r") as file:
    dataset = json.load(file)
file.close()

In [19]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

In [22]:
from langchain_community.llms import Ollama
llm = Ollama(model='qwen:14b')

In [23]:
llm.invoke("Hello")

'Hello there! How can I assist you today?\n'

In [24]:
embeddings = OllamaEmbeddings()

In [25]:
import joblib

doc_label = joblib.load('./vec_prompt/doc_label')
pred_labels = joblib.load('./vec_prompt/pred_labels')
true_labels = joblib.load('./vec_prompt/true_labels')

In [26]:
exp_preds = pred_labels[0]
exp_preds

['PREAMBLE']

In [27]:
sample_query = dataset[0]['annotations'][0]['result'][0]['value']['text']
sample_label = dataset[0]['annotations'][0]['result'][0]['value']['labels']
sample_query, sample_label

('      IN THE HIGH COURT OF KARNATAKA,\n          CIRCUIT BENCH AT GULBARGA\n\nDATED THIS THE 22ND DAY OF FEBRUARY, 2013',
 ['PREAMBLE'])

In [14]:
# from langchain_core.prompts import ChatPromptTemplate
# prompt = ChatPromptTemplate.from_messages([
#     ("system", """You are a specialized system focused on semantic annotation of court opinion."""),
#     ("user", """\n RHETORICAL ROLE:
# Rhetorical roles in legal writing refer to the distinct functions or purposes that different parts of a document, such as
# a legal opinion, serve in conveying information, persuading the reader, and constructing a coherent argument. These
# roles encompass various elements like factual background, legal principles, arguments, counter arguments, and
# conclusions, each contributing to the document's overall persuasive and informative structure.
# \n labelling TASK:
# Your task is to label each sentence in the document with one of the following predefined rhetorical roles: ['Preamble', 'Facts',
# 'Ruling by Lower Court', 'Issues', 'Argument by Petitioner', 'Argument by Respondent', 'Analysis', 'Statute',
# 'Precedent Relied', 'Precedent Not Relied', 'Ratio of the decision', 'Ruling by Present Court', 'NONE'].
# Assign the role that best describes the purpose or function of each sentence in the context of the legal opinion.
# Your output should just be the role as string.
# Here is the sentence you have to do predictions for: 
# {current_sent}
# Choose from the following top likely labels:{labels}""")
# ])

In [33]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a specialized system focused on semantic annotation of court opinion."""),
    ("user", """\n RHETORICAL ROLE:
Rhetorical roles in legal writing refer to the distinct functions or purposes that different parts of a document, such as
a legal opinion, serve in conveying information, persuading the reader, and constructing a coherent argument. These
roles encompass various elements like factual background, legal principles, arguments, counter arguments, and
conclusions, each contributing to the document's overall persuasive and informative structure.
\n labelling TASK:
Your task is to label each sentence in the document with one of the following predefined rhetorical roles: ['Preamble', 'Facts',
'Ruling by Lower Court', 'Issues', 'Argument by Petitioner', 'Argument by Respondent', 'Analysis', 'Statute',
'Precedent Relied', 'Precedent Not Relied', 'Ratio of the decision', 'Ruling by Present Court', 'NONE'].

Assign the role that best describes the purpose or function of each sentence in the context of the legal opinion.
Your output should just be the role as string.
Here is the sentence you have to do predictions for: 
{current_sent}
""")
])

In [34]:
chain = prompt | llm

response = chain.invoke({
    'current_sent': sample_query
})

KeyboardInterrupt: 

In [32]:
print(response)

NONE



In [14]:
from tqdm import tqdm

In [17]:
pred_labels[0]

['PREAMBLE']

In [22]:
len(dataset[0]['annotations'][0]['result']), len(pred_labels), len(true_labels)

(91, 91, 91)

In [23]:
llm_labels = []
for i, data in tqdm(enumerate(dataset[0]['annotations'][0]['result']), total=91):
    sample_query = data['value']['text']
    
    
    query_preds = []
    for out in pred_labels[i]:
        query_preds.extend(out)
    
    prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a specialized system focused on semantic annotation of court opinion."""),
    ("user", """\n RHETORICAL ROLE:
Rhetorical roles in legal writing refer to the distinct functions or purposes that different parts of a document, such as
a legal opinion, serve in conveying information, persuading the reader, and constructing a coherent argument. These
roles encompass various elements like factual background, legal principles, arguments, counter arguments, and
conclusions, each contributing to the document's overall persuasive and informative structure.
\n labelling TASK:
Your task is to label each sentence in the document with one of the following predefined rhetorical roles: ['Preamble', 'Facts',
'Ruling by Lower Court', 'Issues', 'Argument by Petitioner', 'Argument by Respondent', 'Analysis', 'Statute',
'Precedent Relied', 'Precedent Not Relied', 'Ratio of the decision', 'Ruling by Present Court', 'NONE'].
Assign the role that best describes the purpose or function of each sentence in the context of the legal opinion.
Your output should just be the role as string.
Here is the sentence you have to do predictions for: 
{current_sent}
Choose from the following top likely labels:{labels}""")
])
    
    chain = prompt | llm

    response = chain.invoke({
        'current_sent': sample_query,
        'labels': pred_labels[0]
    })
    llm_labels.append(response.strip())
    

100%|██████████| 91/91 [00:16<00:00,  5.55it/s]


In [24]:
len(llm_labels)

91

In [28]:
llm_labels[:10], true_labels[:10]

(['PREAMBLE',
  'Preamble',
  'Preamble',
  'PREAMBLE',
  'PREAMBLE',
  'Facts',
  'Facts',
  'Facts',
  'Facts',
  'Facts'],
 ['PREAMBLE',
  'PREAMBLE',
  'PREAMBLE',
  'PREAMBLE',
  'NONE',
  'FAC',
  'FAC',
  'FAC',
  'FAC',
  'FAC'])

In [36]:
label_abbrv = {
    'preamble': 'PREAMBLE',
    'facts': 'FAC',
    'ruling by lower court': 'RLC',
    'issues': 'ISSUE',
    'argument by petitioner': 'ARG_PETITIONER',
    'argument by respondent': 'ARG_RESPONDENT',
    'analysis': 'ANALYSIS',
    'statute ': 'STA',
    'precedent relied': 'PRE_RELIED',
    'precedent not relied': 'PRE_NOT_RELIED',
    'ratio of the decision': 'Ratio',
    'ruling by present court': 'RPC',
    'none': 'NONE'
}

In [30]:
for i in range(91):
    llm_labels[i] = llm_labels[i].lower()
    true_labels[i] = true_labels[i].lower()

llm_labels[:10], true_labels[:10]

(['preamble',
  'preamble',
  'preamble',
  'preamble',
  'preamble',
  'facts',
  'facts',
  'facts',
  'facts',
  'facts'],
 ['preamble',
  'preamble',
  'preamble',
  'preamble',
  'none',
  'fac',
  'fac',
  'fac',
  'fac',
  'fac'])

In [48]:
llm_label_abbrv = []
for i, label in enumerate(llm_labels):
    try:
        label = label.replace("'", "")
        llm_label_abbrv.append(label_abbrv[label].lower())
    except:
        llm_label_abbrv.append('none')

In [49]:
len(llm_label_abbrv)

91

In [50]:
llm_label_abbrv[:10], true_labels[:10]

(['preamble',
  'preamble',
  'preamble',
  'preamble',
  'preamble',
  'fac',
  'fac',
  'fac',
  'fac',
  'fac'],
 ['preamble',
  'preamble',
  'preamble',
  'preamble',
  'none',
  'fac',
  'fac',
  'fac',
  'fac',
  'fac'])

In [51]:
from sklearn.metrics import accuracy_score, f1_score
acc = accuracy_score(true_labels, llm_label_abbrv)
f1 = f1_score(true_labels, llm_label_abbrv, average="macro")

In [52]:
acc, f1

(0.2967032967032967, 0.22656693981557866)

In [54]:
llm_label_abbrv, true_labels

(['preamble',
  'preamble',
  'preamble',
  'preamble',
  'preamble',
  'fac',
  'fac',
  'fac',
  'fac',
  'fac',
  'fac',
  'fac',
  'arg_petitioner',
  'preamble',
  'fac',
  'fac',
  'fac',
  'preamble',
  'fac',
  'fac',
  'preamble',
  'fac',
  'issue',
  'fac',
  'preamble',
  'arg_petitioner',
  'fac',
  'fac',
  'arg_petitioner',
  'fac',
  'arg_petitioner',
  'fac',
  'fac',
  'fac',
  'arg_petitioner',
  'arg_petitioner',
  'none',
  'arg_respondent',
  'arg_petitioner',
  'preamble',
  'fac',
  'fac',
  'arg_petitioner',
  'preamble',
  'fac',
  'none',
  'fac',
  'fac',
  'issue',
  'arg_petitioner',
  'analysis',
  'fac',
  'analysis',
  'analysis',
  'issue',
  'preamble',
  'preamble',
  'preamble',
  'preamble',
  'preamble',
  'analysis',
  'fac',
  'preamble',
  'fac',
  'arg_petitioner',
  'analysis',
  'analysis',
  'analysis',
  'arg_petitioner',
  'issue',
  'issue',
  'preamble',
  'preamble',
  'none',
  'preamble',
  'none',
  'preamble',
  'arg_petitioner',
 